<a href="https://colab.research.google.com/github/LucasBezerraSantos/Alura_Machine_Learning/blob/master/NLP_Classificacao_Tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Trabalhando com NLP

-----------

A importância do Processamento de Linguagem Natural (NLP) em técnicas de Machine Learning reside na capacidade de lidar com dados textuais e extrair significado e insights a partir deles. O NLP permite que os algoritmos compreendam, interpretem e processem o texto de maneira similar aos humanos, o que é essencial para lidar com grandes volumes de dados textuais, como redes sociais, documentos, e-mails, entre outros. Abaixo aplicaremos um pequeno estudo de caso, realizando classificaçõs de Tags para perguntas realizadas no **stack overflow**.

### Leitura e reconhecimento dos dados

In [1]:
import pandas as pd
import numpy as np

In [2]:
dados = pd.read_csv('/content/sample_data/stackoverflow_perguntas.csv')

In [3]:
dados.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


In [4]:
dados.describe()

,Perguntas,Tags
count,5408,5408
unique,5384,37
top,CODE,jquery
freq,19,1320


### Manipulação de dados

O código a seguir faz a iteração entre cada linha do data series, depois separa onde houver mais de uma palavra, e adiciona na lista caso ainda não tenha sido importada. Com isso, conseguimos verificar todas as tags unicas do conjunto de dados.

In [5]:
lista = list()
for tags in dados.Tags.unique():
  for tag in tags.split():
    if tag not in lista:
      lista.append(tag)

print(lista)

['node.js', 'jquery', 'html', 'angular']


Essa função tem o objetivo de criar colunas no DF para cada TAG. 
Recebe a lista de tags unicas - identificada pelo código anterior - e itera sobre cada linha do DF retornando 1 quando estiver contida na coluna TAGS e 0 quando não.  

In [6]:
def codigo_tag(lista):
    for item in lista:
        dados[item] = [1 if item in tag else 0 for tag in dados.Tags]
    return dados.head()

In [7]:
codigo_tag(lista)

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


A função ZIP reune os valores das colunas criadas e agrupa em uma nova coluna que nomeamos "todas_as_tags"

In [8]:
lista_tgs_zip = list(
    zip(dados['node.js'], dados['jquery'], dados['html'], dados['angular'])
    )
dados['todas_as_tags'] = lista_tgs_zip

In [9]:
dados.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_as_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


### TfidfVectorizer

**A função TfidfVectorizer do sklearn** é utilizada para transformar um conjunto de documentos de texto em uma matriz numérica, representando a importância relativa de cada palavra nos documentos. O nome "Tfidf" vem da abreviação de "Term Frequency-Inverse Document Frequency", que é uma medida estatística comumente usada em NLP.

A função TfidfVectorizer combina os processos de tokenização, contagem de frequência de termos e aplicação da fórmula Tfidf em uma única etapa. Ela atribui um valor numérico a cada palavra com base em sua frequência no documento e sua relevância em relação a outros documentos do conjunto.

Essa técnica é útil em problemas de aprendizado de máquina em que a representação textual é necessária, como classificação de documentos, agrupamento de textos, recomendação de conteúdo, análise de sentimentos, entre outros. A matriz resultante do TfidfVectorizer é uma representação numérica que pode ser usada como entrada para algoritmos de aprendizado de máquina, permitindo que eles trabalhem com texto de maneira eficiente.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

Separação em traino e teste

In [11]:
perguntas_train, perguntas_test, tags_train, tags_test = train_test_split( 
    dados.Perguntas, dados.todas_as_tags, test_size=0.30, random_state=427 )

In [12]:
vetorizar = TfidfVectorizer(max_features=5000, max_df=0.85)
vetorizar.fit(dados.Perguntas)
perguntas_train_tfidf = vetorizar.transform(perguntas_train)
perguntas_test_tfidf = vetorizar.transform(perguntas_test)

In [13]:
print(perguntas_train_tfidf.shape)
print(perguntas_test_tfidf.shape)

(3785, 5000)
(1623, 5000)


### OneVsRestClassifier

A função OneVsRestClassifier do sklearn é utilizada para lidar com problemas de classificação multiclasse através da estratégia "One-vs-Rest" (um-contra-todos). Essa estratégia é aplicada quando temos um problema de classificação com mais de duas classes.

A ideia por trás do OneVsRestClassifier é treinar um classificador binário para cada classe do problema, onde cada classificador é responsável por distinguir uma classe específica das demais. Durante o treinamento, as amostras correspondentes à classe em questão são consideradas como positivas, enquanto as amostras das outras classes são consideradas como negativas.

Quando é necessário fazer uma previsão para uma nova amostra, todos os classificadores binários são utilizados e a classe com a maior probabilidade de pertencer a ela é selecionada como a classe final da amostra.

Essa abordagem é especialmente útil quando o conjunto de dados é desbalanceado, ou seja, quando algumas classes têm um número muito maior de amostras do que outras. O OneVsRestClassifier permite lidar com esse desequilíbrio, treinando um classificador para cada classe independentemente, o que pode levar a melhores resultados em comparação com abordagens de classificação direta.

In [14]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [15]:
regressao_logistica = LogisticRegression() 
classificador = OneVsRestClassifier(regressao_logistica) 

# para que funcione corretamente essa técnica exige um algoritmo de classificação como parâmetro.

In [16]:
tags_train_array = np.asarray(list(tags_train))
tags_test_array = np.asarray(list(tags_test))

# o formato deve ser alterado de "data series para array". A função list remove as tuplas e mantém um array de listas.

In [17]:
classificador.fit(perguntas_train_tfidf, tags_train_array)
resultado = classificador.score(perguntas_test_tfidf, tags_test_array)
print('Acurácia: {0:.2f}%'.format(resultado * 100))

Acurácia: 38.88%


Uma breve comparação de resultados: são 13 tags possíveis entre as combinações, se déssemos 1 palpite a chance de acerto é de 7,69 %. O algoritmo acerta em 38,88% dos casos.

In [18]:
dados.todas_as_tags.unique()

array([(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 1, 1),
       (0, 0, 0, 1), (0, 1, 1, 0), (1, 0, 0, 1), (0, 1, 1, 1),
       (0, 1, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0), (1, 0, 1, 1),
       (1, 1, 1, 0)], dtype=object)

In [19]:
len(dados.todas_as_tags.unique())

13

In [20]:
palpite = 1/13
print(f' {round(palpite * 100, 2)}% ')

 7.69% 


**A métrica hamming_loss do sklearn** é uma medida de desempenho utilizada para avaliar a qualidade de modelos de classificação multilabel. Essa métrica calcula a fração média de rótulos incorretamente previstos para uma amostra de dados.

A hamming_loss compara os rótulos previstos pelo modelo com os rótulos verdadeiros das amostras de teste. Para cada amostra, a métrica verifica se todos os rótulos previstos correspondem aos rótulos verdadeiros. Se houver pelo menos uma discrepância, a amostra é considerada incorretamente classificada.

A hamming_loss retorna um valor entre 0 e 1, onde um valor mais próximo de 0 indica um melhor desempenho do modelo, pois representa uma menor taxa de erro na predição dos rótulos. Um valor de 1 indica que todas as amostras foram classificadas incorretamente.

Essa métrica é especialmente útil quando se lida com problemas de classificação multilabel, nos quais uma amostra pode ser associada a múltiplas classes simultaneamente. A hamming_loss permite avaliar a capacidade do modelo em prever corretamente todas as classes relevantes para uma amostra, considerando a sobreposição e interseção dos rótulos.

In [21]:
from sklearn.metrics import hamming_loss

In [22]:
previsao_classificador = classificador.predict(perguntas_test_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_test_array, previsao_classificador)
print('hamming_loss: {0:.2f}'.format(hamming_loss_onevsrest))

hamming_loss: 0.20


### BinaryRelevance

**A função BinaryRelevance do skmultilearn** é uma abordagem para lidar com problemas de classificação multilabel, nos quais cada amostra pode pertencer a várias classes simultaneamente. Essa função permite transformar o problema multilabel em vários problemas de classificação binária independentes.

Em vez de tentar prever diretamente todos os rótulos para uma amostra, a abordagem BinaryRelevance cria um classificador binário separado para cada rótulo. Cada classificador binário é treinado para prever a presença ou ausência de um rótulo específico, sem levar em consideração os outros rótulos.

Durante a etapa de treinamento, a função BinaryRelevance ajusta cada classificador binário aos dados de treinamento, criando um conjunto de modelos independentes. Durante a etapa de predição, os modelos são aplicados separadamente a cada amostra para determinar a presença ou ausência de cada rótulo.

A função BinaryRelevance é útil quando se deseja utilizar algoritmos de classificação binária existentes para lidar com problemas multilabel. Ela permite que esses algoritmos sejam aplicados de forma independente para cada rótulo, sem considerar a correlação entre eles.

In [24]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 5.3 MB/s eta 0:00:00


In [25]:
from skmultilearn.problem_transform import BinaryRelevance

In [26]:
classificador_multi_br = BinaryRelevance(regressao_logistica)
classificador_multi_br.fit(perguntas_train_tfidf, tags_train_array)
resultado = classificador_multi_br.score(perguntas_test_tfidf, tags_test_array)
print('Acurácia: {0:.2f}%'.format(resultado * 100))

Acurácia: 38.88%


In [27]:
previsao_classificador = classificador_multi_br.predict(perguntas_test_tfidf)
hamming_loss_multi_br = hamming_loss(tags_test_array, previsao_classificador)
print('hamming_loss: {0:.2f}'.format(hamming_loss_multi_br))

hamming_loss: 0.20


### ClassifierChain

**A função ClassifierChain do skmultilearn** é outra abordagem para lidar com problemas de classificação multilabel. Assim como a BinaryRelevance, ela também transforma o problema multilabel em uma série de problemas de classificação binária.

A diferença principal é que a ClassifierChain leva em consideração a dependência entre os rótulos ao criar a sequência de classificadores. Cada classificador é treinado para prever a presença ou ausência de um rótulo específico, levando em conta as informações dos rótulos anteriores na sequência.

Durante a etapa de treinamento, a função ClassifierChain ajusta cada classificador binário aos dados de treinamento, considerando a presença dos rótulos anteriores na sequência. Isso permite que cada classificador aprenda a partir das informações dos rótulos anteriores, capturando a dependência entre eles.

Durante a etapa de predição, os modelos são aplicados em sequência para cada amostra. O resultado da predição de um classificador é utilizado como entrada para o próximo classificador na sequência, levando em conta a dependência entre os rótulos.

A abordagem ClassifierChain é útil quando há uma forte dependência entre os rótulos e é importante capturar essa dependência para obter um melhor desempenho na classificação multilabel. No entanto, ela pode ser mais computacionalmente intensiva do que a BinaryRelevance, uma vez que cada classificador na sequência depende das predições dos classificadores anteriores.

In [28]:
from skmultilearn.problem_transform import ClassifierChain

In [29]:
classificador_multi = ClassifierChain(regressao_logistica)
classificador_multi.fit(perguntas_train_tfidf, tags_train_array)
resultado = classificador_multi.score(perguntas_test_tfidf, tags_test_array)
print('Acurácia: {0:.2f}%'.format(resultado * 100))

Acurácia: 48.37%


In [30]:
previsao_classificador = classificador_multi.predict(perguntas_test_tfidf)
hamming_loss_multi = hamming_loss(tags_test_array, previsao_classificador)
print('hamming_loss: {0:.2f}'.format(hamming_loss_multi))

hamming_loss: 0.22
